Run all cell before working! (Cell-->Run All)

In [20]:
#import packages
# NumPy is a package for scientific computing with Python which contains various functions and tools. 
import numpy as np  
import matplotlib.pyplot as plt
from ipywidgets import widgets,Layout
from IPython.display import display,clear_output
from IPython.display import HTML
%run ModelList.ipynb
%matplotlib inline

In [21]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Monte Carlo Method

The Monte Carlo method is a technique to propagate uncertainties through models. When model input parameters (E modulus, spring constant,...) are modeled as random variables to take uncertainties into account, one is interested in analyzing the effect on model outputs (stress, strain, temperature, velocity of the system at a speciﬁc point in time and/or space). Model input random variable are denoted as $\mathbf{Y}$, the function $g$ represents the model and $Z=g(\mathbf{Y})$ refers to the model output. Note that the function $g$ is often not available in closed-form and requires the solution of a differential equation. 

One way to quantify the output uncertainty is to compute moments such as the mean value and the variance

\begin{align*}
\mathbb{E}[g(\mathbf{Y})] &= \int_{\Gamma} g(\mathbf{y})f_\mathbf{Y} (\mathbf{y}) \,dy, \\
\mathbb{V}[g(\mathbf{Y})] &= \int_{\Gamma}(g(\mathbf{y}) - \mathbb{E}[g(\mathbf{Y})] )^2 f_\mathbf{Y}(\mathbf{y})\,dy,
\end{align*}

where $\Gamma$ denotes the support of the random vector $\mathbf{Y}$ and $f_\mathbf{Y}$ its density. 

The main principles of the Monte Carlo method can be explained by considering the mean value only, hence, we will focus on the approximation of $\mu = \mathbb{E}[g(\mathbf{Y})]$ in the following. 

The Monte Carlo method (MC in the following) is a very simple, direct and robust technique which may be even used for some models with discontinuous response $g$. We introduce $\{\mathbf{Y}^{(k)}\}_{k=1}^K$, where $\mathbf{Y}^{(k)} \sim \mathbf{Y}$ represent independent and identically distributed random variables. In particular, $\mathbf{Y}^{(k)}$ models the outcome of the $k$-th independent draw of $\mathbf{Y}$, which is realized by a random number generator. $\{\mathbf{Y}^{(k)}\}_{k=1}^K$ is called the sample and $\mathbf{Y}^{(k)}$ a sample point. For each sample point one can then call the model function $g$ to compute the output sample $\{Z^{(k)} = g(\mathbf{Y}^{(k)})\}_{k=1}^K$. This procedure can be viewed as an experiment on a computer, which is repeated $K$ times (independently) to obtain some information on the underlying stochastic behavior. The MC estimator is then defined as
$$\mu \approx \tilde{\mu}_K =  \frac{1}{K}\sum_{k=1}^{K} Z^{(k)},$$
representing the output sample arithmetic mean. 

It should be noted that although $\mu$ is fixed, the MC estimator $\tilde{\mu}_K$ is random, i.e., the estimated value may change when the experiment is repeated. This is indicated already by the name estimator, which in statistics is a tool to approximate an unknown parameter based on experimental data. Repitition of the experiment will lead to different results, both in a real and in the MC computer experiment. 

If the function to be approximated is not the mean value, but a quantity given as 

$$ \Psi(\mathbf{Y}) = \int_{\Gamma} \psi(\mathbf{y}) f_\mathbf{Y} (\mathbf{y}) \,dy,$$

the MC method can be used in a similar way; the resulting approximation simply reads 

$$ \Psi(\mathbf{Y}) = \int_{\Gamma} \psi(\mathbf{y}) f_\mathbf{Y} (\mathbf{y}) \,dy \approx \frac{1}{K}\sum_{k=1}^{K} \psi(\mathbf{Y}^{(k)}),$$

however, one should be aware that often the factor in front of the summation must be adapted to obtain an unbiased estimate. 

The root-mean-square error of the MC method is given as 

$$ RMSE = \frac{\sigma_g}{\sqrt{K}},$$

where $\sigma_g^2 = \mathbb{V}[g(\mathbf{Y})]$, which means that the error decreases for an increasing sample size $K$.

### Estimation of $\pi$

An example to illustrate the MC principles is the numerical approximation of $\pi$. There holds

$$\pi = 4\int_{[0,1]} \int_{[0,1]} \chi_{r(\mathbf{y})\leq 1} f_\mathbf{Y}(\mathbf{y}) \mathrm{d}y_1 \mathrm{d}y_2  \approx \frac{4}{K} \sum_{i=1}^K \chi_{r(\mathbf{y}^{(i)})\leq 1},$$

where $\chi_{r(\mathbf{y}^{(i)})\leq 1}$ is equal to $1$ if $\sqrt{(y_1^{(i)})^2 + (y_2^{(i)})^2} \leq 1$ and $0$ otherwise. Intuitively, this means that we draw $K$ points in the square $[0,1] \times [0,1]$ and distinguish between the points inside of the unit circle (in red below) and the points outside of the unit circle (in blue below). The ratio of points inside of the circle vs. the total number of points is then proportional to $\pi$. The result is more and more accurate if $K$ becomes larger. You can carry out the experiment below with a variable number of sample points and check the associated RMSE.

Slide the bar to vary the number of sample points.

In [26]:
#Slide the bar to change the number of samples
N=widgets.IntSlider(value=2000,min=1000,max=50000,step=1000,continuous_update=False)
S=widgets.HBox([widgets.Label('N:'),N])
out=widgets.interactive_output(PiEstimate,{'N':N})
display(S,out)

Output()


### Monte Carlo confidence interval:

Since the MC approximation is itself random, it is important to provide some level of confidence to the estimated value $\tilde{\mu}_K$. A confidence level provides a bound $\tilde{\mu}_K \pm \Delta$ which would contain the true value $\mu$ in with $95,98,99,...$ (the confidence level) of the cases, if the experiment would be repeated many times. It can be shown that, for instance, the interval associated to a 95 percent confidence level is obtained as $\Delta = 1.96\sigma_{g} /\sqrt{K}$, where $\sigma_g$ can also be approximated by the Monte Carlo method to obtain a numerical value. 

In the example below, confidence intervals are computed for the approximation of $\pi$. On can clearly see the decreasing confidence bounds for a larger sample size, as expected from theory. 

In [23]:
#Convergence Study
out1=widgets.Output() 
def clicked1(_):
    global mu_MC, sig_MC, Nvalues
    with out1: 
     mu_MC, sig_MC, Nvalues = Convergence(N1.value)           
def clicked2(_):
   with out1:
    clear_output()
    PlotConvergence(mu_MC,sig_MC, Nvalues)
N1=widgets.IntText(value=4000)
S=widgets.HBox([N1])   
B1=widgets.Button(description='Run')
B2=widgets.Button(description='Plot Convergence')
B1.on_click(clicked1)
B2.on_click(clicked2)
display(widgets.Label('Enter Sample Size'),S,widgets.Label('Click Run to compute mean and standard deviation of the samples and Plot Convergence to see the results'),B1,B2,out1)    

Label(value='Enter Sample Size')

Label(value='Click Run to compute mean and standard deviation of the samples and Plot Convergence to see the r…

Button(description='Run', style=ButtonStyle())

Button(description='Plot Convergence', style=ButtonStyle())

Output()

### Physical Examples

We now apply the Monte Carlo method to physical examples. 

The first test case is a harmonic oscillator, represented by the differential equation

$$m\frac{d^2}{dt^2}{x(t)} + \gamma \frac{dx}{dt} + kx = fcos(\omega t),$$
$$x(0) = x_o, \frac{dx}{dt}(0) = x_1$$

where,
-  m is the mass
-  $\gamma$ is the damping coefficient
-  k is the spring constant
-  f is the amplitude of the force
-  $x_o$ is the initial displacement
-  $x_1$ is the initial velocity, are the inputs of the system.
-  $\omega$ is the frequency of the excitation force.

All input random variables are collected in a random vector $\textbf{Y} = 
\begin{bmatrix}
    \gamma  \\
    k\\
    x_o \\
    x_1 \\
    \omega \\
\end{bmatrix}$. 

It is assumed that each $Y_i$ is uniformly distributed with a $10\%$ variation from their nominal values $Y_{i,o}$ i.e.  $Y_{i,o}$$\sim$ (0.9 $Y_{i,o}$,1.1 $Z_{Y,o}$).

In [24]:
# Setting the nominal values for the input parameters 
out2=widgets.Output()
#Harmonic Ocillator
def clicked1(_):
   with out2: 
    clear_output()
    HarOci(gamma_0.value,cb1.value,k_0.value,cb2.value,f_0.value,cb3.value,omega_0.value,cb4.value,x0_0.value,cb5.value,x1_0.value,cb6.value,variation.value,sampleSize.value)
def clicked2(_):
    with out2:
     clear_output()
     Cant_Beam(Length.value,cb1.value,Width.value,cb2.value,Load.value,cb3.value,Material.value,cb4.value,variation.value,sampleSize.value)
gamma_0 = widgets.FloatText(value=0.1,description='$\gamma$')   
k_0 = widgets.FloatText(value=0.035,description='$k$')     
f_0 = widgets.FloatText(value=0.1,description='$f$') 
omega_0 = widgets.FloatText(value=1,description='$\omega$') 
x0_0 = widgets.FloatText(value=0.5,description='$x_{o}$') 
x1_0 =widgets.FloatText(value=0,description='$x_{1}$')
variation=widgets.IntText(value=10,description='Variation(%)')
sampleSize=widgets.IntSlider(value=100,min=50,max=200,step=50,description='Sample Size')
B1=widgets.Button(description='Run')
B1.on_click(clicked1)
cb1=widgets.Checkbox(value=False,description='Uncertain',disabled=False)
cb2=widgets.Checkbox(value=False,description='Uncertain',disabled=False)
cb3=widgets.Checkbox(value=False,description='Uncertain',disabled=False)
cb4=widgets.Checkbox(value=False,description='Uncertain',disabled=False)
cb5=widgets.Checkbox(value=False,description='Uncertain',disabled=False)
cb6=widgets.Checkbox(value=False,description='Uncertain',disabled=False)
h1 = widgets.HBox([gamma_0,cb1])
h2 = widgets.HBox([k_0,cb2])
h3 = widgets.HBox([f_0,cb3])
h4 = widgets.HBox([omega_0,cb4])
h5 = widgets.HBox([x0_0,cb5])
h6 = widgets.HBox([x1_0,cb6])
C1=widgets.VBox([h1,h2,h3,h4,h5,h6,B1])
C2=widgets.VBox([variation,sampleSize])
tab1=widgets.HBox(children=[C1,C2])

#Cantilever Beam
Length = widgets.FloatText(value=1,description='Beam Length')
Width = widgets.FloatText(value=0.01,description='Beam Side')
Load = widgets.FloatText(value=100,description='End Load')
Material = widgets.FloatText(value=6.9e9,description='E')
B2=widgets.Button(description='Run')
B2.on_click(clicked2)
i1 = widgets.HBox([Length,cb1])
i2 = widgets.HBox([Width,cb2])
i3 = widgets.HBox([Load,cb3])
i4 = widgets.HBox([Material,cb4])
C3 = widgets.VBox([i1,i2,i3,i4,B2])
tab2=widgets.HBox([C3,C2])

tab=widgets.Tab(children=[tab1,tab2])
tab.set_title(0,'Harmonic Ocillator')
tab.set_title(1,'Cantilever Beam')
display(tab,out2)

Output()

## Shortcomings of Monte carlo method

Since this method involves evaluating the model at a large number of sample points, the computational cost can quickly become to expensive. Also, the rate of convergence ${O}(K^{-1/2})$ is rather slow. Hence, when applicable, surrogate modeling techniques should beof MC method is very slow of the order used to reduce the computational effort. An alternative consists in using variance reduction techniques or Quasi-Monte Carlo methods.

In [25]:
# import ipynb.fs.full.MonteCarloHarmonicOscillator 

# References

-  Lecture notes Methods of Uncertainty Analysis and Quantification

-  The basics of python language are described here https://www.scipy-lectures.org/intro/language/python_language.html

-  For the jupyter notebook installation you can see https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/install.html
